In [1]:
import pandas as pd
import pysam
import os

In [2]:
# inputs
mut_path = "/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/merged_unique_variants.bed"
nuc_path = "/home/ubuntu/honors_research/peak_align/edited_TSS_all_adj+1Nuc_250bp.bed" #expanded bed file
mutations = pd.read_csv(mut_path, sep="\t", header=None)
nucleosomes = pd.read_csv(nuc_path, sep="\t", header = None)
nucleosomes.columns = ["chrom", "start", "end", "NAME", "score", "strand", "distance_to_nuc", "phase"]


In [3]:
# peak align files
phase0 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase0.cdt", sep="\t")
phase1 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase1.cdt", sep="\t")
phase2 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase2.cdt", sep="\t")
phase3 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase3.cdt", sep="\t")
phase4 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase4.cdt", sep="\t")
phase5 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase5.cdt", sep="\t")
phase6 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase6.cdt", sep="\t")
phase7 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase7.cdt", sep="\t")
phase8 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase8.cdt", sep="\t")
phase9 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase9.cdt", sep="\t")
phase10 = pd.read_csv("/home/ubuntu/honors_research/peak_align/phased_haplotype_muts/phase10.cdt", sep="\t")

# merging all peak align files into one big df
phases = [phase0, phase1, phase2, phase3, phase4, phase5, phase6, phase7, phase8, phase9, phase10]
merged_phases = pd.concat(phases)
merged_phases

,NAME,0,1,2,3,4,5,6,7,8,...,240,241,242,243,244,245,246,247,248,249
0,chr1_21782419_21782419,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr16_68085718_68085718,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr1_148634329_148634329,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr1_156747365_156747365,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr1_172449989_172449989,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055,chr18_57435408_57435408,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6056,chr19_37251877_37251877,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6057,chr8_1701428_1701428,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6058,chr13_108215535_108215535,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def nuc_phases(phases, nucleosomes):
    merged_phases = pd.merge(phases, nucleosomes[["chrom", "start", "NAME"]], 
                  on="NAME", how="left")
    
    # move chrom and start after NAME
    merged_phases = merged_phases[["chrom", "start", "NAME"] + list(merged_phases.columns[1:-2])]

    # dropping NAME column
    merged_phases = merged_phases.drop(columns=["NAME"])

    return merged_phases

In [5]:
merged_phases = nuc_phases(merged_phases, nucleosomes)

In [6]:
merged_phases

,chrom,start,0,1,2,3,4,5,6,7,...,240,241,242,243,244,245,246,247,248,249
0,chr1,21782294,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr16,68085593,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr1,148634204,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr1,156747240,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr1,172449864,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51311,chr18,57435283,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51312,chr19,37251752,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51313,chr8,1701303,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51314,chr13,108215410,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def exposed_muts(merged_phases, mutations):
    
    # Subsetting merged_phases: first two columns, and the rest
    nucleosomes_columns = merged_phases.iloc[:, 0:1]
    mutations_columns = merged_phases.iloc[:, 1:]

    #if length of mutations columns / 2 ends in 5 (e.g. 250/2 = 125)
    

    if len(mutations_columns.columns)==251: #i.e. expanded by 250bp
        columns_to_select = []
        for col in range(mutations_columns.shape[1]):
            if (col) % 10 == 0:  # Check if the column index minus 4 is a multiple of 10
                columns_to_select.extend([col, col+1, col+2])
        if len(columns_to_select) > 251:
            columns_to_select = columns_to_select[:252]
    
    
    
    else: #i.e. expanded by 500bp
        columns_to_select = [0]
        for col in range(mutations_columns.shape[1]):
            if (col-4) % 10 == 0:  # Check if the column index minus 4 is a multiple of 10
                columns_to_select.extend([col, col+1, col + 2])
    
    print(columns_to_select)


    # Ensure column indices are valid (within bounds)
    columns_to_select = [col for col in columns_to_select if 0 <= col < mutations_columns.shape[1]]

    # combining nucleosomes_columns and mutation_columns (of columns_to_select)
    merged_phases_filtered = pd.concat([nucleosomes_columns, mutations_columns.iloc[:, columns_to_select]], axis=1)
    merged_phases_filtered = merged_phases_filtered[merged_phases_filtered.iloc[:, 2:].sum(axis=1) != 0] # filter out rows where all selected columns are 0

   # exposure status
    filtered_mutations = mutations.copy()
    filtered_mutations["exposed"] = "not_exposed"

    accum = 0
    for idx, row in merged_phases_filtered.iterrows():
        start = int(row[1])
        chrom = row[0]

        
        for col in row.index[2:]:
            value = row[col]
            if value != 0:
                added = int(col)
                pos = start + added
                # print(pos)

                # matching
                matching = filtered_mutations[
                    (filtered_mutations[0] == chrom) &
                    (filtered_mutations[1] == pos)
                ]
                
                # if matching -> put "exposed" in a new column in mutations df

                if not matching.empty:
                    filtered_mutations.loc[
                        (filtered_mutations[0] == chrom) & (filtered_mutations[1] == pos), "exposed"
                    ] = "exposed"
                    accum += 1


    return filtered_mutations


exposed_mutations = exposed_muts(merged_phases, mutations)

[1, 2, 3, 11, 12, 13, 21, 22, 23, 31, 32, 33, 41, 42, 43, 51, 52, 53, 61, 62, 63, 71, 72, 73, 81, 82, 83, 91, 92, 93, 101, 102, 103, 111, 112, 113, 121, 122, 123, 131, 132, 133, 141, 142, 143, 151, 152, 153, 161, 162, 163, 171, 172, 173, 181, 182, 183, 191, 192, 193, 201, 202, 203, 211, 212, 213, 221, 222, 223, 231, 232, 233, 241, 242, 243, 251, 252, 253]


/tmp/ipykernel_1266966/3594759366.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start = int(row[1])
/tmp/ipykernel_1266966/3594759366.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  chrom = row[0]


KeyboardInterrupt: 

In [9]:
exposed_mutations

,0,1,2,exposed
0,chr1,86377,86378,not_exposed
1,chr1,139232,139233,not_exposed
2,chr1,817185,817186,not_exposed
3,chr1,825766,825767,not_exposed
4,chr1,905372,905373,not_exposed
...,...,...,...,...
193019,chrY,56823143,56823144,not_exposed
193020,chrY,56823146,56823147,not_exposed
193021,chrY,56858714,56858715,not_exposed
193022,chrY,56862954,56862955,not_exposed


In [10]:
exposed_mutations.groupby("exposed").size().reset_index(name='count')


,exposed,count
0,exposed,772
1,not_exposed,192252
